In [1]:
import warnings
import pprint
import skrebate
import imblearn
import time
from imblearn import under_sampling, over_sampling, combine
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn import (preprocessing, svm, linear_model, ensemble, naive_bayes,
                    tree, neighbors, decomposition, kernel_approximation, cluster)
from sklearn.pipeline import Pipeline

from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import (KFold, GroupKFold, StratifiedKFold,
                                    LeaveOneGroupOut, cross_validate,
                                    cross_val_predict, learning_curve, GridSearchCV)
from sklearn.feature_selection import SelectKBest, f_regression, SelectFromModel, VarianceThreshold, f_classif
from sklearn.metrics import r2_score, auc, roc_auc_score, balanced_accuracy_score, confusion_matrix, roc_curve
from sklearn import metrics
from sklearn.preprocessing import QuantileTransformer, quantile_transform
from xgboost import XGBRegressor, XGBClassifier

warnings.simplefilter('ignore')

In [2]:
import os
import sys
import numpy as np
import pandas as pd
import re


import plotly.plotly as py
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [3]:
from mlxtend.regressor import StackingCVRegressor, StackingRegressor
from mlxtend.classifier import StackingCVClassifier, StackingClassifier

In [4]:
Galaxy_ML = os.path.dirname(os.path.dirname(os.getcwd()))

sys.path.append(os.path.join(Galaxy_ML, 'galaxy_ml'))
try:
    from iraps_classifier import BinarizeTargetClassifier, BinarizeTargetRegressor
except ImportError:
    raise ImportError("galaxy_ml is not in path!")


In [5]:
import subprocess
subprocess.call('pip freeze', shell=True)

0

In [6]:
Olararib_1017 = pd.read_csv(os.path.join(Galaxy_ML, 'galaxy_ml/test-data/Olaparib_1017.tsv.gz'), sep='\t', index_col=0)
Olararib_1017

,depmap_ID,DRUG_ID,DRUG_NAME,Primary Disease,Gender,AUC,TSPAN6 (ENSG00000000003),TNMD (ENSG00000000005),DPM1 (ENSG00000000419),SCYL3 (ENSG00000000457),...,NUDT3 (ENSG00000272325),C6ORF165 (ENSG00000272514),MUSTN1 (ENSG00000272573),DOC2B (ENSG00000272636),RP11-309L24.9 (ENSG00000272899),C2ORF15 (ENSG00000273045),GRIN2B (ENSG00000273079),SNURF (ENSG00000273173),RP11-1263C18.1 (ENSG00000273238),ZBTB8B (ENSG00000273274)
5,ACH-000087,1017,Olaparib,Bone Cancer,Male,0.846324,4.955127,0.070389,5.873567,2.853996,...,4.694880,0.137504,0.545968,3.203201,0.978196,0.669027,0.000000,7.092229,0.669027,0.042644
6,ACH-000327,1017,Olaparib,Lung Cancer,Female,0.970014,4.041769,0.000000,5.813525,1.918386,...,3.730096,0.028569,0.070389,0.000000,1.035624,2.310340,0.000000,0.641546,0.000000,0.000000
7,ACH-000905,1017,Olaparib,Bladder Cancer,Male,0.915457,5.881175,0.000000,6.643135,2.039138,...,2.664483,0.084064,0.226509,0.000000,0.422233,2.003602,0.000000,5.660210,0.000000,0.432959
8,ACH-000242,1017,Olaparib,Bladder Cancer,Male,0.987345,7.496574,0.000000,6.458940,2.364572,...,3.658783,0.739848,0.201634,0.000000,0.378512,3.557042,0.000000,6.843607,0.000000,0.948601
9,ACH-000384,1017,Olaparib,Bladder Cancer,Female,0.952230,5.755155,0.000000,6.928844,2.895303,...,3.324811,2.042644,1.531069,0.000000,0.622930,3.244887,0.000000,5.405312,0.000000,1.077243
13,ACH-000323,1017,Olaparib,Brain Cancer,Male,0.922807,4.590362,0.000000,6.647890,1.899176,...,3.187451,0.111031,0.526069,0.000000,0.298658,0.176323,0.000000,6.463524,0.650765,0.014355
14,ACH-000137,1017,Olaparib,Brain Cancer,Female,0.901520,5.108106,0.000000,6.888500,2.207893,...,3.861955,0.137504,0.879706,0.000000,0.333424,1.490570,0.056584,7.145372,0.000000,0.757023
15,ACH-000738,1017,Olaparib,Brain Cancer,Male,0.901069,4.078951,0.000000,6.691953,2.121015,...,3.056584,0.084064,0.855990,0.000000,0.536053,0.505891,0.014355,6.101818,0.000000,0.704872
16,ACH-000571,1017,Olaparib,Brain Cancer,Male,0.960333,3.408712,0.000000,6.677931,2.269033,...,3.247928,0.124328,0.000000,0.000000,0.731183,0.495695,0.000000,0.669027,0.000000,0.028569
17,ACH-000040,1017,Olaparib,Brain Cancer,Male,0.983662,3.828835,0.000000,6.436295,2.073820,...,3.782409,0.526069,0.516015,0.000000,1.021480,0.687061,0.014355,6.302685,0.000000,0.000000


In [7]:
X, y = Olararib_1017.iloc[:, 6:].values, Olararib_1017.iloc[:, 5].values.ravel()
data1 = go.Scatter(
        y = y,
        mode = "markers",
        name = "y_ture",
)

layout = dict(
    xaxis=dict(
        title='Sample No.'
    ),
    yaxis=dict(
        title='Value'
    ),
    title="Distribution of target y"
    
)
fig = go.Figure(data=[data1], layout=layout)
iplot(fig)

# To show plot, paste the link to this GitHub notebook into http://nbviewer.jupyter.org/

In [8]:
# Shared testing parameters
from iraps_classifier import IRAPSCore, IRAPSClassifier
from iraps_classifier import binarize_auc_scorer, binarize_average_precision_scorer
from iraps_classifier import BinarizeTargetTransformer
from model_validations import OrderedKFold, RepeatedOrderedKFold

cv = OrderedKFold(n_splits=5, shuffle=True, random_state=123)

scoring = {
    'ap': binarize_average_precision_scorer,
    'auc': binarize_auc_scorer
}

# Tree algorithms

In [34]:
# RandomForestClassifier
model = ensemble.RandomForestClassifier(random_state=0, class_weight="balanced")
estimator = BinarizeTargetClassifier(model)

rval = rval_rfc = cross_validate(estimator, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([0.86160016, 0.85600901, 0.85827184, 0.85818601, 0.8872931 ]),
  'score_time': array([0.01232386, 0.01170301, 0.01711082, 0.01590991, 0.01007795]),
  'test_ap': array([0.13490267, 0.14858907, 0.09597874, 0.09858755, 0.12532637]),
  'train_ap': array([0.99868421, 0.99857752, 1.        , 1.        , 0.99117351]),
  'test_auc': array([0.49784946, 0.72576832, 0.51950355, 0.48387097, 0.64184397]),
  'train_auc': array([0.99992964, 0.99992793, 1.        , 1.        , 0.99929637])},
 0.12067688101574414,
 0.5737672538702052)

In [33]:
# RandomForestRegressor
model = ensemble.RandomForestRegressor(random_state=0)
estimator = BinarizeTargetRegressor(model)

rval = rval_rfr = cross_validate(estimator, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([37.2194407 , 37.08532286, 38.15170002, 44.18254685, 23.02951121]),
  'score_time': array([0.01522994, 0.01655984, 0.0124321 , 0.01369405, 0.0097177 ]),
  'test_ap': array([0.193065  , 0.27374859, 0.11484267, 0.23472446, 0.08524331]),
  'train_ap': array([0.88710804, 0.84430983, 0.87371068, 0.89847941, 0.9155982 ]),
  'test_auc': array([0.63333333, 0.64775414, 0.46690307, 0.64623656, 0.46926714]),
  'train_auc': array([0.98592739, 0.98032432, 0.98601802, 0.98560252, 0.98853082])},
 0.18032480387682087,
 0.572698848470983)

In [39]:
# RandomForestRegressor
model = XGBClassifier(booster='gbtree')
estimator = BinarizeTargetClassifier(model)

rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([95.18019509, 95.06028199, 82.76291513, 82.86060071, 66.92184901]),
  'score_time': array([0.20289898, 0.18975186, 0.17431092, 0.17913127, 0.1443491 ]),
  'test_ap': array([0.1825592 , 0.20934824, 0.23201331, 0.15369977, 0.11308134]),
  'train_ap': array([1., 1., 1., 1., 1.]),
  'test_auc': array([0.62473118, 0.75177305, 0.6855792 , 0.61290323, 0.56501182]),
  'train_auc': array([1., 1., 1., 1., 1.])},
 0.1781403725005992,
 0.6479996949592008)

In [40]:
# RandomForestRegressor
model = XGBRegressor(booster='dart')
estimator = BinarizeTargetRegressor(model)

rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([78.1516099 , 78.51624084, 73.64381695, 74.016469  , 62.3948679 ]),
  'score_time': array([0.22927213, 0.1422174 , 0.12280393, 0.14198899, 0.11951995]),
  'test_ap': array([0.18876325, 0.46920738, 0.08357605, 0.18640097, 0.3403279 ]),
  'train_ap': array([1.        , 0.9965505 , 0.99928876, 0.99315768, 0.99863272]),
  'test_auc': array([0.49784946, 0.71276596, 0.45744681, 0.66451613, 0.61820331]),
  'train_auc': array([1.        , 0.99963964, 0.99992793, 0.9992111 , 0.99985927])},
 0.25365510902236943,
 0.5901563334095935)

In [49]:
# RandomForestRegressor
model = XGBClassifier(booster='dart')
estimator = BinarizeTargetClassifier(model)

rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

/Users/guq/miniconda3/envs/python3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:700: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.

/Users/guq/miniconda3/envs/python3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:700: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.



({'fit_time': array([54.3279388 , 54.78639698, 58.14463997, 58.99958086, 49.04657722]),
  'score_time': array([0.21970987, 0.12079382, 0.11057305, 0.116395  , 0.09666681]),
  'test_ap': array([0.1825592 , 0.20934824, 0.23201331, 0.15369977, 0.11308134]),
  'train_ap': array([1., 1., 1., 1., 1.]),
  'test_auc': array([0.62473118, 0.75177305, 0.6855792 , 0.61290323, 0.56501182]),
  'train_auc': array([1., 1., 1., 1., 1.])},
 0.1781403725005992,
 0.6479996949592008)

In [41]:
# RandomForestRegressor
model = XGBRegressor(booster='gblinear')
estimator = BinarizeTargetRegressor(model)

rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([26.35266781, 26.27872992, 25.77276421, 25.12663984, 22.47579813]),
  'score_time': array([0.14111519, 0.12460303, 0.09760785, 0.11496401, 0.08929992]),
  'test_ap': array([0.19652783, 0.17541398, 0.54640689, 0.29896941, 0.17979842]),
  'train_ap': array([0.99932524, 0.99514322, 0.99579975, 0.9962882 , 0.99868421]),
  'test_auc': array([0.60107527, 0.59456265, 0.77777778, 0.5655914 , 0.58628842]),
  'train_auc': array([0.99992964, 0.9994955 , 0.99956757, 0.99960555, 0.99985927])},
 0.27942330633236634,
 0.6250591016548463)

In [50]:
# RandomForestRegressor
model = XGBClassifier(booster='gblinear')
estimator = BinarizeTargetClassifier(model)

rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([24.78723407, 24.74860811, 19.673738  , 19.70348072, 17.64688706]),
  'score_time': array([0.10551882, 0.08257294, 0.07710195, 0.07884502, 0.07354999]),
  'test_ap': array([0.25236326, 0.27912091, 0.35086208, 0.25377759, 0.14262643]),
  'train_ap': array([1., 1., 1., 1., 1.]),
  'test_auc': array([0.74623656, 0.73758865, 0.71276596, 0.70537634, 0.58392435]),
  'train_auc': array([1., 1., 1., 1., 1.])},
 0.2557500560204116,
 0.6971783726073363)

In [43]:
# RandomForestRegressor
model = svm.LinearSVR()
estimator = BinarizeTargetRegressor(model)

rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([16.90042114, 16.46488619, 16.54236293, 14.97077203, 15.11325002]),
  'score_time': array([0.00956583, 0.01592088, 0.00743699, 0.00809598, 0.00716376]),
  'test_ap': array([0.1570016 , 0.14810019, 0.33751344, 0.28619806, 0.11887029]),
  'train_ap': array([1., 1., 1., 1., 1.]),
  'test_auc': array([0.55483871, 0.51536643, 0.76241135, 0.50860215, 0.46453901]),
  'train_auc': array([1., 1., 1., 1., 1.])},
 0.20953671522572775,
 0.561151529017006)

In [44]:
# RandomForestRegressor
model = svm.SVR()
estimator = BinarizeTargetRegressor(model)

rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([1.73863626, 1.87055707, 1.59330297, 1.47868395, 1.28396702]),
  'score_time': array([0.42028475, 0.51183677, 0.32850575, 0.26224804, 0.26199985]),
  'test_ap': array([0.20222199, 0.11302112, 0.16017414, 0.29686041, 0.14140872]),
  'train_ap': array([0.28641164, 0.32144943, 0.31205531, 0.33792667, 0.37808371]),
  'test_auc': array([0.68172043, 0.49408983, 0.678487  , 0.5311828 , 0.5070922 ]),
  'train_auc': array([0.63580073, 0.7045045 , 0.67120721, 0.70087437, 0.71882916])},
 0.1827372746350011,
 0.5785144513078625)

In [45]:
# RandomForestRegressor
model = linear_model.LinearRegression()
estimator = BinarizeTargetRegressor(model)

rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

/Users/guq/miniconda3/envs/python3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:700: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.



({'fit_time': array([3.58205414, 3.94731593, 4.23961806, 4.07637525, 2.50525331]),
  'score_time': array([0.0103519 , 0.00686216, 0.01069283, 0.009197  , 0.00714993]),
  'test_ap': array([0.1627808 , 0.22433335, 0.55330589, 0.30477076, 0.11665522]),
  'train_ap': array([1., 1., 1., 1., 1.]),
  'test_auc': array([0.57849462, 0.58392435, 0.80496454, 0.58494624, 0.43498818]),
  'train_auc': array([1., 1., 1., 1., 1.])},
 0.27236920317010826,
 0.5974635857545947)

In [46]:
# RandomForestRegressor
model = linear_model.Ridge()
estimator = BinarizeTargetRegressor(model)

rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([0.76888299, 0.767272  , 0.79688692, 0.79752707, 0.94955897]),
  'score_time': array([0.00853682, 0.00872016, 0.01020217, 0.01008582, 0.00823402]),
  'test_ap': array([0.16114061, 0.22433335, 0.55609063, 0.3046532 , 0.11637032]),
  'train_ap': array([1., 1., 1., 1., 1.]),
  'test_auc': array([0.57096774, 0.58392435, 0.80732861, 0.58387097, 0.43380615]),
  'train_auc': array([1., 1., 1., 1., 1.])},
 0.2725176225260262,
 0.5959795622664531)

In [48]:
# RandomForestRegressor
model = linear_model.LassoLars()
estimator = BinarizeTargetRegressor(model)

rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([0.63219404, 0.63213992, 0.61962914, 0.6197722 , 0.55905485]),
  'score_time': array([0.00810504, 0.00832486, 0.0084908 , 0.0083487 , 0.00580716]),
  'test_ap': array([0.09708738, 0.08737864, 0.08737864, 0.09708738, 0.08737864]),
  'train_ap': array([0.09223301, 0.08980583, 0.08980583, 0.09951456, 0.09223301]),
  'test_auc': array([0.5, 0.5, 0.5, 0.5, 0.5]),
  'train_auc': array([0.5, 0.5, 0.5, 0.5, 0.5])},
 0.0912621359223301,
 0.5)

# Stacking

In [52]:
li_xgbc = BinarizeTargetClassifier(XGBClassifier(booster='gblinear'))
li_xgbr = XGBRegressor(booster='gblinear')
li_regr = linear_model.LinearRegression()
dart_xgbr = XGBRegressor(booster='gbtree'))
tree_xgbc = BinarizeTargetClassifier(XGBClassifier(booster='gbtree')
ridge = BinarizeTargetRegressor(linear_model.Ridge())

estimator = StackingCVRegressor(regressors=[li_xgbc, li_xgbr, li_regr, dart_xgbr], meta_regressor=ridge)
rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([493.7193048 , 494.2635498 , 474.86757517, 474.83508897,
         361.44782996]),
  'score_time': array([0.23650002, 0.24748325, 0.45393896, 0.74201679, 0.23339796]),
  'test_ap': array([0.22202179, 0.26082275, 0.26540851, 0.24967395, 0.14351668]),
  'train_ap': array([1., 1., 1., 1., 1.]),
  'test_auc': array([0.69032258, 0.65602837, 0.58865248, 0.58064516, 0.54728132]),
  'train_auc': array([1., 1., 1., 1., 1.])},
 0.22828873426373525,
 0.6125859833752765)

In [55]:
li_xgbc = BinarizeTargetClassifier(XGBClassifier(booster='gblinear'))
li_xgbr = BinarizeTargetRegressor(XGBRegressor(booster='gblinear'))
li_regr = BinarizeTargetRegressor(linear_model.LinearRegression())
dart_xgbr = BinarizeTargetRegressor(XGBRegressor(booster='gbtree'))
tree_xgbc = BinarizeTargetClassifier(XGBClassifier(booster='gbtree'))
ridge = BinarizeTargetRegressor(linear_model.Ridge())

estimator = StackingRegressor(regressors=[li_xgbc, li_xgbr, li_regr, dart_xgbr], meta_regressor=ridge)
rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([96.9130609 , 97.29991007, 96.83809209, 96.69504523, 90.05896711]),
  'score_time': array([0.27100301, 0.25398421, 0.24240375, 0.25119376, 0.219244  ]),
  'test_ap': array([0.19653881, 0.29207242, 0.45725215, 0.30045446, 0.13980685]),
  'train_ap': array([1., 1., 1., 1., 1.]),
  'test_auc': array([0.59462366, 0.65130024, 0.78486998, 0.57526882, 0.50118203]),
  'train_auc': array([1., 1., 1., 1., 1.])},
 0.27722493693512895,
 0.6214489437962328)

In [57]:
li_xgbc = BinarizeTargetClassifier(XGBClassifier(booster='gblinear'))
li_xgbr = BinarizeTargetRegressor(XGBRegressor(booster='gblinear'))



estimator = StackingCVRegressor(regressors=[li_xgbc, li_xgbr], meta_regressor=li_xgbc, cv=cv)
rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([191.8877182 , 190.62075114, 185.74918485, 185.33830881,
         122.59476876]),
  'score_time': array([0.37178493, 0.1400919 , 0.1320312 , 0.31230688, 0.12863994]),
  'test_ap': array([0.20350183, 0.18619149, 0.41742316, 0.26587717, 0.14943181]),
  'train_ap': array([1., 1., 1., 1., 1.]),
  'test_auc': array([0.62150538, 0.60756501, 0.76359338, 0.57741935, 0.56973995]),
  'train_auc': array([1., 1., 1., 1., 1.])},
 0.2444850923755666,
 0.627964615267292)

In [59]:
li_xgbc = BinarizeTargetClassifier(XGBClassifier(booster='gblinear'))
li_xgbr = BinarizeTargetRegressor(XGBRegressor(booster='gblinear'))



estimator = StackingRegressor(regressors=[li_xgbc, li_xgbr], meta_regressor=li_xgbc)
rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([37.99209094, 38.07731581, 38.842978  , 38.91868496, 35.52078199]),
  'score_time': array([0.13653493, 0.13990808, 0.14602399, 0.16231012, 0.12697101]),
  'test_ap': array([0.24110587, 0.25961513, 0.36868437, 0.23965035, 0.14482645]),
  'train_ap': array([1., 1., 1., 1., 1.]),
  'test_auc': array([0.71075269, 0.6678487 , 0.75295508, 0.63978495, 0.55673759]),
  'train_auc': array([1., 1., 1., 1., 1.])},
 0.2507764345339895,
 0.6656158011133989)

In [60]:
li_xgbc = BinarizeTargetClassifier(XGBClassifier(booster='gblinear'))
li_xgbr = BinarizeTargetRegressor(XGBRegressor(booster='gblinear'))



estimator = StackingRegressor(regressors=[li_xgbc, li_xgbr], meta_regressor=li_xgbr)
rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([38.89744902, 38.96073127, 39.31595802, 39.38810515, 35.56646299]),
  'score_time': array([0.14418197, 0.14413476, 0.143852  , 0.16138387, 0.12565899]),
  'test_ap': array([0.2279173 , 0.25644716, 0.38504068, 0.24236346, 0.14548832]),
  'train_ap': array([1., 1., 1., 1., 1.]),
  'test_auc': array([0.66666667, 0.63356974, 0.75531915, 0.61935484, 0.55910165]),
  'train_auc': array([1., 1., 1., 1., 1.])},
 0.2514513839759419,
 0.6468024098223136)

In [61]:
li_xgbc = BinarizeTargetClassifier(XGBClassifier(booster='gblinear'))
li_xgbr = BinarizeTargetRegressor(XGBRegressor(booster='gblinear'))



estimator = StackingRegressor(regressors=[li_xgbc, li_xgbr], meta_regressor=li_regr)
rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([39.24416208, 39.20689201, 41.59895134, 40.93226004, 35.78442383]),
  'score_time': array([0.15409493, 0.15156507, 0.14003086, 0.13279796, 0.13306808]),
  'test_ap': array([0.19652783, 0.17541398, 0.54640689, 0.29896941, 0.17979842]),
  'train_ap': array([0.99799169, 0.99357611, 0.99138715, 0.99573449, 0.99589245]),
  'test_auc': array([0.60107527, 0.59456265, 0.77777778, 0.5655914 , 0.58628842]),
  'train_auc': array([0.99978891, 0.99935135, 0.99913514, 0.99953981, 0.99957782])},
 0.27942330633236634,
 0.6250591016548463)

In [62]:
li_xgbc = BinarizeTargetClassifier(XGBClassifier(booster='gblinear'))
li_xgbr = BinarizeTargetRegressor(XGBRegressor(booster='gblinear'))



estimator = StackingRegressor(regressors=[li_xgbr, li_xgbc], meta_regressor=li_regr)
rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([38.75615072, 38.71743274, 39.71388292, 39.61611295, 35.56258488]),
  'score_time': array([0.14720821, 0.14346433, 0.16296315, 0.17165327, 0.12340307]),
  'test_ap': array([0.19652783, 0.17541398, 0.54640689, 0.29896941, 0.17979842]),
  'train_ap': array([0.99799169, 0.99357611, 0.99138715, 0.99573449, 0.99589245]),
  'test_auc': array([0.60107527, 0.59456265, 0.77777778, 0.5655914 , 0.58628842]),
  'train_auc': array([0.99978891, 0.99935135, 0.99913514, 0.99953981, 0.99957782])},
 0.27942330633236634,
 0.6250591016548463)

In [77]:
li_xgbc = BinarizeTargetClassifier(XGBClassifier(booster='gblinear'))
li_xgbr = BinarizeTargetRegressor(XGBRegressor(booster='gblinear'))

svr = BinarizeTargetRegressor(svm.SVR())

estimator = StackingRegressor(regressors=[li_xgbr, li_xgbc], meta_regressor=svr)
rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([39.40429688, 39.24081731, 40.81403899, 40.93233109, 36.58354902]),
  'score_time': array([0.18129921, 0.15313983, 0.14315319, 0.15432191, 0.13195682]),
  'test_ap': array([0.20160965, 0.2123033 , 0.4738413 , 0.26515902, 0.16573177]),
  'train_ap': array([1., 1., 1., 1., 1.]),
  'test_auc': array([0.61182796, 0.60756501, 0.76832151, 0.56989247, 0.57683215]),
  'train_auc': array([1., 1., 1., 1., 1.])},
 0.2637290092721102,
 0.6268878212460917)

In [85]:
li_xgbr = XGBRegressor(booster='gblinear')
dart_xgbr = XGBRegressor(booster='dart')

li_svr = BinarizeTargetRegressor(svm.LinearSVR())

estimator = StackingRegressor(regressors=[li_xgbr, dart_xgbr], meta_regressor=li_svr, store_train_meta_features=True)
estimator.fit(X, y)

/Users/guq/miniconda3/envs/python3/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



StackingRegressor(meta_regressor=BinarizeTargetRegressor(less_is_positive=True,
            regressor=LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
     intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
     random_state=None, tol=0.0001, verbose=0),
            value=None, z_score=-1),
         refit=True,
         regressors=[XGBRegressor(base_score=0.5, booster='gblinear', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='..., reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)],
         store_train_meta_features=True, use_features_in_secondary=False,
         verbose=0)

In [10]:
li_xgbr = XGBRegressor(booster='gblinear')
dart_xgbr = XGBRegressor(booster='dart')

b_li_xgbr = BinarizeTargetRegressor(li_xgbr)

estimator = StackingCVRegressor(regressors=[li_xgbr], meta_regressor=b_li_xgbr, use_features_in_secondary=True, cv=cv)
rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([124.69434381, 125.30925369, 133.40533686, 133.12793589,
          92.99674702]),
  'score_time': array([0.15308809, 0.18115306, 0.14425492, 0.32195592, 0.16678977]),
  'test_ap': array([0.09960013, 0.15663279, 0.16775422, 0.15215827, 0.14618425]),
  'train_ap': array([0.19825039, 0.13646325, 0.14589986, 0.18380555, 0.17153999]),
  'test_auc': array([0.49784946, 0.53546099, 0.51891253, 0.58064516, 0.6713948 ]),
  'train_auc': array([0.66260906, 0.61167568, 0.63445045, 0.63572415, 0.58858711])},
 0.14446593288746462,
 0.5608525890337833)

In [102]:
li_xgbr = XGBRegressor(booster='gblinear')
dart_xgbr = XGBRegressor(booster='dart')

b_li_xgbr = BinarizeTargetRegressor(li_xgbr)

estimator = StackingCVRegressor(regressors=[dart_xgbr], meta_regressor=b_li_xgbr, use_features_in_secondary=True, cv=cv)
rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([311.69929695, 310.30883503, 287.00330901, 286.14769006,
         196.48079991]),
  'score_time': array([0.37614608, 0.17628908, 0.18121386, 0.3412571 , 0.19308019]),
  'test_ap': array([0.19652783, 0.17541398, 0.54640689, 0.29896941, 0.17979842]),
  'train_ap': array([0.99932524, 0.99514322, 0.99579975, 0.9962882 , 0.99868421]),
  'test_auc': array([0.60107527, 0.59456265, 0.77777778, 0.5655914 , 0.58628842]),
  'train_auc': array([0.99992964, 0.9994955 , 0.99956757, 0.99960555, 0.99985927])},
 0.27942330633236634,
 0.6250591016548463)

In [9]:
li_xgbr = XGBRegressor(booster='gblinear')
li_xgbc = BinarizeTargetClassifier(XGBClassifier(booster='gblinear'))

b_li_xgbr = BinarizeTargetRegressor(li_xgbr)

estimator = StackingCVRegressor(regressors=[li_xgbc], meta_regressor=b_li_xgbr, use_features_in_secondary=True, cv=cv)
rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([136.05898094, 136.12344909, 128.66250396, 128.56200123,
          84.30865002]),
  'score_time': array([0.15638518, 0.16479206, 0.22243094, 0.22582483, 0.14675403]),
  'test_ap': array([0.09960628, 0.15651334, 0.16701266, 0.15190823, 0.14657013]),
  'train_ap': array([0.19796009, 0.13624807, 0.14594732, 0.18203343, 0.17135979]),
  'test_auc': array([0.49784946, 0.53427896, 0.51654846, 0.57849462, 0.67375887]),
  'train_auc': array([0.66225725, 0.61117117, 0.63466667, 0.6348695 , 0.58830566])},
 0.14432212846543951,
 0.5601860748875161)

In [11]:
li_xgbr = XGBRegressor(booster='gblinear')
li_xgbc = BinarizeTargetClassifier(XGBClassifier(booster='gblinear'))

logistic = BinarizeTargetClassifier(linear_model.LogisticRegression(random_state=10))

estimator = StackingRegressor(regressors=[li_xgbc, b_li_xgbr], meta_regressor=logistic)
rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([39.61985326, 39.70125604, 39.80138016, 39.68585801, 35.44212008]),
  'score_time': array([0.143152  , 0.15950108, 0.14376307, 0.1465857 , 0.12894583]),
  'test_ap': array([0.23996342, 0.26059812, 0.3677766 , 0.24239712, 0.14467946]),
  'train_ap': array([1., 1., 1., 1., 1.]),
  'test_auc': array([0.71290323, 0.6749409 , 0.75059102, 0.64516129, 0.55437352]),
  'train_auc': array([1., 1., 1., 1., 1.])},
 0.25108294316947316,
 0.6675939906962556)

In [12]:
li_xgbr = XGBRegressor(booster='gblinear')
li_xgbc = BinarizeTargetClassifier(XGBClassifier(booster='gblinear'))

tree_xgbc = BinarizeTargetClassifier(XGBClassifier())

estimator = StackingRegressor(regressors=[li_xgbc, b_li_xgbr], meta_regressor=tree_xgbc)
rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([46.16073012, 46.32835817, 44.60787392, 44.5616641 , 35.26880884]),
  'score_time': array([0.23214793, 0.15381193, 0.14202785, 0.14490581, 0.13124299]),
  'test_ap': array([0.20155797, 0.14340511, 0.21885522, 0.2148291 , 0.12292945]),
  'train_ap': array([1., 1., 1., 1., 1.]),
  'test_auc': array([0.71827957, 0.59869976, 0.67257683, 0.58924731, 0.55141844]),
  'train_auc': array([1., 1., 1., 1., 1.])},
 0.18031536840502105,
 0.6260443834362847)

# Feature_selection

In [18]:
from sklearn.feature_selection import SelectKBest, f_classif

kbest = SelectKBest(score_func=f_classif, k=100)
pipe = Pipeline([('kbest', kbest), ('est', XGBClassifier(booster='gblinear'))])

estimator = BinarizeTargetClassifier(pipe)


#estimator = StackingRegressor(regressors=[li_xgbc, b_li_xgbr], meta_regressor=tree_xgbc)
rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([0.87585616, 0.87273693, 0.81873202, 0.8245759 , 0.70295906]),
  'score_time': array([0.01136398, 0.01154304, 0.00921082, 0.00909805, 0.00841069]),
  'test_ap': array([0.15505833, 0.43227028, 0.4143363 , 0.3067487 , 0.1697656 ]),
  'train_ap': array([0.52671955, 0.52214137, 0.50793002, 0.50172871, 0.58561999]),
  'test_auc': array([0.6688172 , 0.82269504, 0.64893617, 0.73225806, 0.643026  ]),
  'train_auc': array([0.83528005, 0.85585586, 0.85772973, 0.84530932, 0.87904588])},
 0.2956358397947346,
 0.703146495843819)

In [40]:
from sklearn.feature_selection import SelectKBest, f_classif, f_regression

kbest = SelectKBest(score_func=f_regression, k=200)
pipe = Pipeline([('kbest', kbest), ('est', XGBRegressor(booster='gblinear'))])

estimator = BinarizeTargetRegressor(pipe)


#estimator = StackingRegressor(regressors=[li_xgbc, b_li_xgbr], meta_regressor=tree_xgbc)
rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([0.82664585, 0.82316613, 1.03288221, 1.00101972, 0.73297596]),
  'score_time': array([0.00999093, 0.00954771, 0.01163602, 0.0094161 , 0.00898194]),
  'test_ap': array([0.15640674, 0.2224037 , 0.11251846, 0.29284852, 0.17181898]),
  'train_ap': array([0.53791992, 0.44624182, 0.46128298, 0.50887466, 0.44218562]),
  'test_auc': array([0.60537634, 0.60165485, 0.5177305 , 0.51935484, 0.65130024]),
  'train_auc': array([0.82620321, 0.78724324, 0.84079279, 0.84636119, 0.82233324])},
 0.19119927941731876,
 0.5790833523983834)

In [51]:
from sklearn.feature_selection import SelectKBest, f_classif, f_regression

kbest = SelectKBest(score_func=f_classif, k=100)
pipe = Pipeline([('kbest', kbest), ('est', XGBClassifier(booster='gblinear'))])

estimator = BinarizeTargetClassifier(pipe)
cv2 = RepeatedOrderedKFold(n_splits=10, n_repeats=4)

#estimator = StackingRegressor(regressors=[li_xgbc, b_li_xgbr], meta_regressor=tree_xgbc)
rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv2, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([0.84537888, 0.84544516, 0.82959485, 0.82854795, 0.83944082,
         0.84481311, 0.85772014, 0.85639191, 0.87500691, 0.86388302,
         0.90355682, 0.90314007, 0.84371805, 0.84578991, 0.85362005,
         0.855304  , 0.85172606, 0.84939003, 0.83908701, 0.84570098,
         0.86185503, 0.84546089, 0.84683514, 0.85004997, 0.84591699,
         0.84661913, 0.87669396, 0.87529325, 0.86442113, 0.86286902,
         0.85207891, 0.86170506, 0.94543314, 0.95135903, 1.018857  ,
         1.04474497, 1.03684878, 1.04905319, 0.99194479, 1.02684999]),
  'score_time': array([0.00786304, 0.0074501 , 0.00714207, 0.00764298, 0.00734377,
         0.00781894, 0.00692606, 0.00716019, 0.00788927, 0.00710702,
         0.00693035, 0.00707579, 0.00713992, 0.00692606, 0.00699687,
         0.00678015, 0.007406  , 0.00706196, 0.00738883, 0.00816894,
         0.01022506, 0.00951004, 0.00713682, 0.00715613, 0.00705218,
         0.00702691, 0.00693202, 0.00685191, 0.00696993, 0.00716114,
      

In [53]:
from sklearn.feature_selection import SelectKBest, f_classif, f_regression, chi2, mutual_info_classif

kbest = SelectKBest(score_func=mutual_info_classif, k=100)
pipe = Pipeline([('kbest', kbest), ('est', XGBClassifier(booster='gblinear'))])

estimator = BinarizeTargetClassifier(pipe)
cv2 = OrderedKFold(n_splits=10)

#estimator = StackingRegressor(regressors=[li_xgbc, b_li_xgbr], meta_regressor=tree_xgbc)
rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv2, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([ 78.91654301,  79.32998705,  76.88240886,  78.37418389,
          75.95218587,  77.74600387,  88.66994882,  89.95894694,
         101.47562003, 102.8608861 ]),
  'score_time': array([0.00601912, 0.00611782, 0.00633407, 0.00654531, 0.00606513,
         0.00589108, 0.00564003, 0.00640726, 0.00574088, 0.0054481 ]),
  'test_ap': array([0.33371594, 0.22287218, 0.25302036, 0.24092419, 0.52846154,
         0.16618687, 0.11347222, 0.63333333, 0.40332675, 0.16696429]),
  'train_ap': array([0.46568576, 0.40693022, 0.4123278 , 0.39498861, 0.36924386,
         0.43190332, 0.41763564, 0.39063657, 0.37261556, 0.39192393]),
  'test_auc': array([0.62318841, 0.59574468, 0.77446809, 0.70638298, 0.80851064,
         0.61304348, 0.52608696, 0.92553191, 0.84574468, 0.7287234 ]),
  'train_auc': array([0.82973621, 0.82545021, 0.80393134, 0.79894808, 0.77457301,
         0.82182352, 0.80991875, 0.79541864, 0.79304897, 0.78966373])},
 0.30622776594284973,
 0.7147425223558435)

In [24]:
from sklearn.feature_selection import SelectKBest, f_classif, f_regression, chi2, mutual_info_classif

kbest = SelectKBest(score_func=f_classif, k=100)
pipe = Pipeline([('kbest', kbest), ('est', XGBClassifier(booster='gblinear'))])

estimator = BinarizeTargetClassifier(pipe)

#estimator = StackingRegressor(regressors=[li_xgbc, b_li_xgbr], meta_regressor=tree_xgbc)
rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([0.80747819, 0.80850697, 0.96671104, 0.9637239 , 0.88724303]),
  'score_time': array([0.00885892, 0.00903296, 0.01237369, 0.01175904, 0.00784993]),
  'test_ap': array([0.15505833, 0.43227028, 0.4143363 , 0.3067487 , 0.1697656 ]),
  'train_ap': array([0.52671955, 0.52214137, 0.50793002, 0.50172871, 0.58561999]),
  'test_auc': array([0.6688172 , 0.82269504, 0.64893617, 0.73225806, 0.643026  ]),
  'train_auc': array([0.83528005, 0.85585586, 0.85772973, 0.84530932, 0.87904588])},
 0.2956358397947346,
 0.703146495843819)

In [25]:
from sklearn.feature_selection import SelectKBest, f_classif, f_regression, chi2, mutual_info_classif

kbest = BinarizeTargetTransformer(SelectKBest(score_func=f_classif, k=100))
b_li_xgbc = BinarizeTargetClassifier(XGBClassifier(booster='gblinear'))
pipe = Pipeline([('kbest', kbest), ('est', b_li_xgbc)])

#estimator = BinarizeTargetClassifier(pipe)

#estimator = StackingRegressor(regressors=[li_xgbc, b_li_xgbr], meta_regressor=tree_xgbc)
rval = rval_xgbc = cross_validate(pipe, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([0.89957118, 0.89838886, 0.77832007, 0.78742003, 0.71401811]),
  'score_time': array([0.0121398 , 0.01203823, 0.01151299, 0.01260591, 0.01006794]),
  'test_ap': array([0.15505833, 0.43227028, 0.4143363 , 0.3067487 , 0.1697656 ]),
  'train_ap': array([0.52671955, 0.52214137, 0.50793002, 0.50172871, 0.58561999]),
  'test_auc': array([0.6688172 , 0.82269504, 0.64893617, 0.73225806, 0.643026  ]),
  'train_auc': array([0.83528005, 0.85585586, 0.85772973, 0.84530932, 0.87904588])},
 0.2956358397947346,
 0.703146495843819)

# preprocessing

In [61]:
from sklearn.feature_selection import SelectKBest, f_classif, f_regression, chi2, mutual_info_classif
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, Normalizer

kbest = SelectKBest(score_func=f_classif, k=100)
scaler = Normalizer(norm='l2')
pipe = Pipeline([('kbest', kbest), ('scaler', scaler), ('est', XGBClassifier(booster='gblinear'))])

estimator = BinarizeTargetClassifier(pipe)
cv2 = OrderedKFold(n_splits=10)

#estimator = StackingRegressor(regressors=[li_xgbc, b_li_xgbr], meta_regressor=tree_xgbc)
rval = rval_xgbc = cross_validate(estimator, X, y, cv=cv2, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([0.86327505, 0.8614881 , 0.86848092, 0.86655998, 0.85222197,
         0.85401702, 0.86022305, 0.85501313, 0.88761592, 0.88791919]),
  'score_time': array([0.00802898, 0.00774574, 0.00735736, 0.00696325, 0.00735092,
         0.00693798, 0.00852895, 0.00747371, 0.00730991, 0.00702381]),
  'test_ap': array([0.39183087, 0.13152668, 0.39108392, 0.38039216, 0.37063492,
         0.17760132, 0.12175806, 0.70486111, 0.31822344, 0.13142691]),
  'train_ap': array([0.40377606, 0.35739313, 0.37260576, 0.37157477, 0.32977998,
         0.3569796 , 0.38050912, 0.32173461, 0.3420138 , 0.37496741]),
  'test_auc': array([0.70652174, 0.57021277, 0.83829787, 0.77021277, 0.8212766 ,
         0.63913043, 0.56086957, 0.94680851, 0.81914894, 0.64361702]),
  'train_auc': array([0.79136691, 0.78156867, 0.77956811, 0.7864495 , 0.77680692,
         0.79129993, 0.79361318, 0.77524825, 0.77025502, 0.78390882])},
 0.31193393945075576,
 0.7316096207215541)

# Stacking

In [42]:
from sklearn.feature_selection import SelectKBest, f_classif, f_regression, chi2, mutual_info_classif

kbest = BinarizeTargetTransformer(SelectKBest(score_func=f_classif, k=100))
li_xgbc = XGBClassifier(booster='gblinear')
b_li_xgbc = BinarizeTargetClassifier(li_xgbc)
tree_xgbr = XGBClassifier(booster='gbtree')
dart_xgbr = XGBClassifier(booster='dart')
li_xgbr = XGBRegressor(booster='gblinear')

logistic = linear_model.LogisticRegression()
stacking = StackingCVRegressor(regressors=[li_xgbr], meta_regressor=b_li_xgbc, cv=cv, use_features_in_secondary=False)

pipe = Pipeline([('kbest', kbest), ('est', BinarizeTargetRegressor(stacking))])

rval = rval_xgbc = cross_validate(pipe, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([1.39193296, 1.43207622, 1.63655019, 1.67825317, 1.43408418]),
  'score_time': array([0.01181793, 0.018718  , 0.01180506, 0.01198483, 0.0103879 ]),
  'test_ap': array([0.07976097, 0.07471882, 0.14323093, 0.09130802, 0.06855856]),
  'train_ap': array([0.05569102, 0.05745081, 0.05488744, 0.05887507, 0.06037589]),
  'test_auc': array([0.36612903, 0.29787234, 0.58628842, 0.44462366, 0.3179669 ]),
  'train_auc': array([0.20071067, 0.2525045 , 0.21466667, 0.18394583, 0.25738812])},
 0.09151545745812409,
 0.40257606954930214)

In [51]:
from sklearn.feature_selection import SelectKBest, f_classif, f_regression, chi2, mutual_info_classif

kbest = BinarizeTargetTransformer(SelectKBest(score_func=f_classif, k=100))
li_xgbc = XGBClassifier(booster='gblinear')
b_li_xgbc = BinarizeTargetClassifier(li_xgbc)
tree_xgbr = XGBClassifier(booster='gbtree')
dart_xgbr = XGBClassifier(booster='dart')
li_xgbr = XGBRegressor(booster='gblinear')

logistic = linear_model.LogisticRegression()
svr = svm.SVR()

cv2 = OrderedKFold(n_splits=10)

stacking = StackingCVClassifier(classifiers=[li_xgbc], meta_classifier=li_xgbc, cv=cv2, use_features_in_secondary=True)

pipe = Pipeline([('kbest', kbest), ('est', BinarizeTargetClassifier(stacking))])

rval = rval_xgbc = cross_validate(pipe, X, y, cv=cv, scoring=scoring, n_jobs=2)
rval, np.mean(rval['test_ap']), np.mean(rval['test_auc'])

({'fit_time': array([1.88816905, 1.8882103 , 1.92051101, 1.92165804, 1.40746522]),
  'score_time': array([0.01359391, 0.01357794, 0.01386905, 0.01347208, 0.01095986]),
  'test_ap': array([0.15505833, 0.43227028, 0.4143363 , 0.3067487 , 0.1697656 ]),
  'train_ap': array([0.52671955, 0.52214137, 0.50793002, 0.50172871, 0.58561999]),
  'test_auc': array([0.6688172 , 0.82269504, 0.64893617, 0.73225806, 0.643026  ]),
  'train_auc': array([0.83528005, 0.85585586, 0.85772973, 0.84530932, 0.87904588])},
 0.2956358397947346,
 0.703146495843819)